In [1]:
# Uncomment lines below for the (verbose) debugging mode
import pycalphad.log
#pycalphad.log.debug_mode()

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from pycalphad import equilibrium
from pycalphad import Database
import pycalphad.variables as v

In [4]:
db_alfe = Database('alfe_sei.TDB')
my_phases_alfe = ['LIQUID', 'FCC_A1', 'HCP_A3', 'AL5FE2', 'AL2FE', 'AL13FE4', 'AL5FE4']
%time eq = equilibrium(db_alfe, ['AL', 'FE', 'VA'] , my_phases_alfe, {v.X('AL'): [0.55], v.T: (300, 1400)}, pdens=100)
print(eq)

(1, 20, 2)
CPU times: user 13.5 s, sys: 10 ms, total: 13.5 s
Wall time: 13.5 s
<xray.Dataset>
Dimensions:  (T: 20, X_AL: 1, phase: 2)
Coordinates:
  * X_AL     (X_AL) float64 0.55
  * T        (T) float64 300.0 357.9 415.8 473.7 531.6 589.5 647.4 705.3 ...
  * phase    (phase) int64 0 1
Data variables:
    points   (X_AL, T, phase) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    MU       (X_AL, T, phase) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    NP       (X_AL, T, phase) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...


In [6]:
eq.nbytes / 2**20

0.0186309814453125